In [6]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *

SOCKET = "wss://stream.binance.com:9443/ws/maticusdt@kline_15m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 75
RSI_OVERSOLD = 20
TRADE_SYMBOL = 'MATICUSD'
TRADE_QUANTITY = 7.5

closes = []
in_position = False
api_key = 'kzzLGVrknr0vg0zU9CAGsGxeFGoKMCRHzJv90fN5FBMiyc2tD4YBW3EEyFr0xKBL'
api_secret = 'uvEpi78O7hvn3GtA6Gq9L9mTe3eP8MilsEbiwdsgsD7Z6nYX945mWB1rJpHu0nk9'

client = Client(api_key, api_secret)

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
received message
{'E': 1623839329760,
 'e': 'kline',
 'k': {'B': '0',
       'L': 110158152,
       'Q': '1828031.85838300',
       'T': 1623839399999,
       'V': '1162427.40000000',
       'c': '1.57266000',
       'f': 110151086,
       'h': '1.58026000',
       'i': '15m',
       'l': '1.56666000',
       'n': 7067,
       'o': '1.57776000',
       'q': '3979627.17523200',
       's': 'MATICUSDT',
       't': 1623838500000,
       'v': '2530873.60000000',
       'x': False},
 's': 'MATICUSDT'}
received message
{'E': 1623839331829,
 'e': 'kline',
 'k': {'B': '0',
       'L': 110158165,
       'Q': '1828944.12471300',
       'T': 1623839399999,
       'V': '1163007.50000000',
       'c': '1.57247000',
       'f': 110151086,
       'h': '1.58026000',
       'i': '15m',
       'l': '1.56666000',
       'n': 7080,
       'o': '1.57776000',
       'q': '3982066.75966200',
       's': 'MATICUSDT',
       't': 1623838500000,
       'v': '2532424.90000000',
       'x': Fal

False